In [69]:
#Import dependencies
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time


In [156]:
#Import datasets
advanced_df = pd.read_csv("resources/advanced.csv")
team_summary_df = pd.read_csv("resources/team_summaries.csv")
player_totals_df = pd.read_csv("resources/player_totals.csv")
team_totals_df = pd.read_csv("resources/team_totals.csv")

In [157]:
#Obtain column headers which contain stat abbreviations
df_cols = [advanced_df.columns, team_summary_df.columns, 
       player_totals_df.columns, team_totals_df.columns]

In [158]:
#List of the four data frames
dfs = [advanced_df, team_summary_df, 
       player_totals_df, team_totals_df]

In [159]:
#Generate list of all column headers for the four dataframes
abbrev = []
for df_col in df_cols:
    for item in df_col:
        if item not in abbrev:
            abbrev.append(item)
abbrev

['seas_id',
 'season',
 'player_id',
 'player',
 'birth_year',
 'pos',
 'age',
 'experience',
 'lg',
 'tm',
 'g',
 'mp',
 'per',
 'ts_percent',
 'x3p_ar',
 'f_tr',
 'orb_percent',
 'drb_percent',
 'trb_percent',
 'ast_percent',
 'stl_percent',
 'blk_percent',
 'tov_percent',
 'usg_percent',
 'ows',
 'dws',
 'ws',
 'ws_48',
 'obpm',
 'dbpm',
 'bpm',
 'vorp',
 'team',
 'abbreviation',
 'playoffs',
 'w',
 'l',
 'pw',
 'pl',
 'mov',
 'sos',
 'srs',
 'o_rtg',
 'd_rtg',
 'n_rtg',
 'pace',
 'e_fg_percent',
 'ft_fga',
 'opp_e_fg_percent',
 'opp_tov_percent',
 'opp_drb_percent',
 'opp_ft_fga',
 'arena',
 'attend',
 'attend_g',
 'gs',
 'fg',
 'fga',
 'fg_percent',
 'x3p',
 'x3pa',
 'x3p_percent',
 'x2p',
 'x2pa',
 'x2p_percent',
 'ft',
 'fta',
 'ft_percent',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts']

In [160]:
#List of all corresponding terms referred to by column header abbreviations
meanings = ['season_id',
 'season',
 'player_id',
 'player',
 'birth_year',
 'position',
 'age',
 'experience',
 'league',
 'team',
 'games',
 'minutes_played',
 'player_efficiency_rating',
 'true_shooting_percentage',
 'three_point_attempt_rate',
 'free_throw_attempt_rate',
 'offensive_rebound_percentage',
 'defensive_rebound_percentage',
 'total_rebound_percentage',
 'assist_percentage',
 'steal_percentage',
 'block_percentage',
 'turnover_percentage',
 'usage_percentage',
 'offensive_win_shares',
 'defensive_win_shares',
 'win_shares',
 'win_shares_per_48_min',
 'offensive_box_plus/minus',
 'defensive_box_plus/minus',
 'box_plus/minus',
 'value_over_replacement_plyr',
 'team',
 'abbreviation',
 'playoffs',
 'wins',
 'losses',
 'pythagorean_wins',
 'pythagorean_losses',
 'margin_of_victory',
 'strength_of_schedule',
 'simple_rating_system',
 'offensive_rating',
 'defensive_rating',
 'net_rating',
 'pace',
 'effective_fieldgoal_percentage',
 'free_throws/fieldgoal_attempts',
 'opponent_effective_fieldgoal_percentage',
 'opponent_turnover_percentage',
 'opponent_defensive_rebound_percentage',
 'opponent_freethrows/fieldgoal_attempts',
 'arena',
 'attend',
 'attend_g',
 'games_started',
 'fieldgoals',
 'fieldgoals_attemped',
 'fieldgoal_percentage',
 'three_points_made',
 'three_point_attempts',
 'three_point_percentage',
 'two_point_made',
 'two_point_attempts',
 'two_point_percentage',
 'freethrows_made',
 'freethrows_attempted',
 'freethrow_percentage',
 'offensive_rebounds',
 'defensive_rebounds',
 'total_rebounds',
 'assists',
 'steals',
 'blocks',
 'turnovers',
 'personal_fouls',
 'points']

In [161]:
#Generate a dictionary of abbreviations (keys) and full terms (values)
nba_gloss = {}
for i in range(len(abbrev)):
    nba_gloss[abbrev[i]] = meanings[i]

In [162]:
#Replace all column headers in dataframes with new headers containing full terms
for df in dfs:
    df.set_axis([nba_gloss[i] for i in list(df.columns)], inplace=True, axis=1)


In [163]:
#Get info on player_totals_df
player_totals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30296 entries, 0 to 30295
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   season_id                       30296 non-null  int64  
 1   season                          30296 non-null  int64  
 2   player_id                       30296 non-null  int64  
 3   player                          30296 non-null  object 
 4   birth_year                      2867 non-null   float64
 5   position                        30296 non-null  object 
 6   age                             30272 non-null  float64
 7   experience                      30296 non-null  int64  
 8   league                          30296 non-null  object 
 9   team                            30296 non-null  object 
 10  games                           30267 non-null  float64
 11  games_started                   21655 non-null  float64
 12  minutes_played                  

In [164]:
#Filter player_totals_df to only include data from 1979-2021
player_totals_filtered_df = player_totals_df.loc[(player_totals_df['season'] < 2022) & 
                                   (player_totals_df['season'] > 1978)]
player_totals_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21999 entries, 649 to 22647
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   season_id                       21999 non-null  int64  
 1   season                          21999 non-null  int64  
 2   player_id                       21999 non-null  int64  
 3   player                          21999 non-null  object 
 4   birth_year                      682 non-null    float64
 5   position                        21999 non-null  object 
 6   age                             21999 non-null  float64
 7   experience                      21999 non-null  int64  
 8   league                          21999 non-null  object 
 9   team                            21999 non-null  object 
 10  games                           21999 non-null  float64
 11  games_started                   20989 non-null  float64
 12  minutes_played                

In [165]:
#Drop ineffective columns from resulting player_totals_filtered_df
player_totals_filtered_df = player_totals_filtered_df.drop([
    'birth_year',
    'three_point_percentage',
    'fieldgoal_percentage',
    'two_point_percentage',
    'effective_fieldgoal_percentage',
    'freethrow_percentage'
], axis=1)
player_totals_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21999 entries, 649 to 22647
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season_id             21999 non-null  int64  
 1   season                21999 non-null  int64  
 2   player_id             21999 non-null  int64  
 3   player                21999 non-null  object 
 4   position              21999 non-null  object 
 5   age                   21999 non-null  float64
 6   experience            21999 non-null  int64  
 7   league                21999 non-null  object 
 8   team                  21999 non-null  object 
 9   games                 21999 non-null  float64
 10  games_started         20989 non-null  float64
 11  minutes_played        21999 non-null  float64
 12  fieldgoals            21999 non-null  float64
 13  fieldgoals_attemped   21999 non-null  float64
 14  three_points_made     21655 non-null  float64
 15  three_point_attem

In [166]:
#Remove remaining rows with nulls in any column from players_totals_filtered_df
player_totals_filtered_df = player_totals_filtered_df.dropna()
player_totals_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20981 entries, 649 to 22280
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season_id             20981 non-null  int64  
 1   season                20981 non-null  int64  
 2   player_id             20981 non-null  int64  
 3   player                20981 non-null  object 
 4   position              20981 non-null  object 
 5   age                   20981 non-null  float64
 6   experience            20981 non-null  int64  
 7   league                20981 non-null  object 
 8   team                  20981 non-null  object 
 9   games                 20981 non-null  float64
 10  games_started         20981 non-null  float64
 11  minutes_played        20981 non-null  float64
 12  fieldgoals            20981 non-null  float64
 13  fieldgoals_attemped   20981 non-null  float64
 14  three_points_made     20981 non-null  float64
 15  three_point_attem

In [167]:
#View team summary dataset
team_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783 entries, 0 to 1782
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   season                                   1783 non-null   int64  
 1   league                                   1783 non-null   object 
 2   team                                     1783 non-null   object 
 3   abbreviation                             1682 non-null   object 
 4   playoffs                                 1783 non-null   bool   
 5   age                                      1719 non-null   float64
 6   wins                                     1697 non-null   float64
 7   losses                                   1697 non-null   float64
 8   pythagorean_wins                         1782 non-null   float64
 9   pythagorean_losses                       1782 non-null   float64
 10  margin_of_victory                        1782 no

In [169]:
#Drop unnecessary columns from team_summary_df
team_summary_filtered_df = team_summary_df[['season',
                                   'league',
                                   'team',
                                   'abbreviation',
                                   'strength_of_schedule',
                                   'playoffs']]
team_summary_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783 entries, 0 to 1782
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1783 non-null   int64  
 1   league                1783 non-null   object 
 2   team                  1783 non-null   object 
 3   abbreviation          1682 non-null   object 
 4   strength_of_schedule  1782 non-null   float64
 5   playoffs              1783 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 71.5+ KB


In [204]:
#Obtain list of team abbreviations:
url = 'https://www.shrpsports.com/nba/explain.htm'
team_abbrev = pd.read_html(url, header=0)
team_abbrev = team_abbrev[4]

In [205]:
#View resulting team_abbrev dataframe
team_abbrev.head()

,List of abbreviations used in other pages:,List of abbreviations used in other pages:.1,List of abbreviations used in other pages:.2
0,Atlanta,Atl,Atlanta Hawks (1968-69 - present)
1,Anderson,And,Anderson Packers (1949-50)
2,Bal Bullets,BlB,Baltimore Bullets (1st team) (1947-48 - 1953-54)
3,Baltimore,Bal,Baltimore Bullets (2nd team) (1963-64 - 1972-73)
4,Boston,Bos,Boston Celtics (1946-47 - present)


In [206]:
#Change column headers of team_abrev dataframe and modify values for column containig full name
teams = team_abbrev['List of abbreviations used in other pages:.2'].apply(lambda x: x.split(' (')[0])
team_abbrev.rename(columns={"List of abbreviations used in other pages:": "Location", 
                            "List of abbreviations used in other pages:.1": "Abbreviation",
                           'List of abbreviations used in other pages:.2':'Full_name'}, inplace=True)
team_abbrev['Full_name'] = teams
team_abbrev.head()

,Location,Abbreviation,Full_name
0,Atlanta,Atl,Atlanta Hawks
1,Anderson,And,Anderson Packers
2,Bal Bullets,BlB,Baltimore Bullets
3,Baltimore,Bal,Baltimore Bullets
4,Boston,Bos,Boston Celtics


In [207]:
#Create dictionary of team abbreviations and full team names from team_abbrev dataframe
team_abbrev_dict ={}

for i in range(len(team_abbrev['Full_name'])):
    team_abbrev_dict[team_abbrev['Full_name'][i]] = team_abbrev['Abbreviation'][i]

In [220]:
#Add four additional entries to team_abbrev_dict
team_abbrev_dict['New York Knicks'] = 'NY'
team_abbrev_dict['Oklahoma City Thunder'] = 'OkC'
team_abbrev_dict['Portland Trail Blazers'] = 'Por'
team_abbrev_dict['League Average'] = 'LGAvg'

In [222]:
#Filter team_summary_df to include years 1979-2021
team_summary_filtered_df = team_summary_filtered_df.loc[(team_summary_filtered_df['season'] < 2022) & 
                                   (team_summary_filtered_df['season'] > 1978)]

In [223]:
#Add updated_abbreviatio column which contains team abbrev from dictionary with team column entries as keys
team_summary_filtered_df['updated_abbreviation'] = [team_abbrev_dict[i] for i in team_summary_filtered_df['team']]
team_summary_filtered_df.drop(['abbreviation'], axis=1, inplace=True)
team_summary_filtered_df.info()

,season,league,team,abbreviation,strength_of_schedule,playoffs,updated_abbreviation
31,2021,NBA,Chicago Bulls,CHI,-0.05,False,Chi
32,2021,NBA,Charlotte Hornets,CHO,-0.01,False,Cha
33,2021,NBA,Cleveland Cavaliers,CLE,0.25,False,Cle
34,2021,NBA,Detroit Pistons,DET,0.09,False,Det
35,2021,NBA,Golden State Warriors,GSW,0.04,False,GS


In [88]:
#View the advanced_df dataset
advanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30296 entries, 0 to 30295
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   season_id                     30296 non-null  int64  
 1   season                        30296 non-null  int64  
 2   player_id                     30296 non-null  int64  
 3   player                        30296 non-null  object 
 4   birth_year                    2867 non-null   float64
 5   position                      30296 non-null  object 
 6   age                           30272 non-null  float64
 7   experience                    30296 non-null  int64  
 8   league                        30296 non-null  object 
 9   team                          30296 non-null  object 
 10  games                         30267 non-null  float64
 11  minutes_played                29162 non-null  float64
 12  player_efficiency_rating      29114 non-null  float64
 13  t

In [231]:
#filter the advanced_df to only include years 1979-2021
advanced_filtered_df =  advanced_df.loc[(advanced_df['season'] < 2022) & 
                                   (advanced_df['season'] > 1978)]
advanced_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21999 entries, 649 to 22647
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   season_id                     21999 non-null  int64  
 1   season                        21999 non-null  int64  
 2   player_id                     21999 non-null  int64  
 3   player                        21999 non-null  object 
 4   birth_year                    682 non-null    float64
 5   position                      21999 non-null  object 
 6   age                           21999 non-null  float64
 7   experience                    21999 non-null  int64  
 8   league                        21999 non-null  object 
 9   team                          21999 non-null  object 
 10  games                         21999 non-null  float64
 11  minutes_played                21999 non-null  float64
 12  player_efficiency_rating      21994 non-null  float64
 13 

In [232]:
#re-generate advanced_filtered_df containing columns of interest
advanced_filtered_df = advanced_filtered_df[['season', 'player', 'player_efficiency_rating',
                                             'true_shooting_percentage', 'three_point_attempt_rate',
                                            'free_throw_attempt_rate', 'offensive_rebound_percentage',
                                            'defensive_rebound_percentage', 'total_rebound_percentage',
                                            'assist_percentage', 'steal_percentage', 'block_percentage',
                                            'usage_percentage', 'offensive_win_shares', 'defensive_win_shares',
                                            'win_shares', 'win_shares_per_48_min', 'offensive_box_plus/minus',
                                            'defensive_box_plus/minus', 'box_plus/minus', 
                                             'value_over_replacement_plyr']]
#Drop rows containing nulls
advanced_filtered_df.dropna(inplace = True)
advanced_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21553 entries, 649 to 22303
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   season                        21553 non-null  int64  
 1   player                        21553 non-null  object 
 2   player_efficiency_rating      21553 non-null  float64
 3   true_shooting_percentage      21553 non-null  float64
 4   three_point_attempt_rate      21553 non-null  float64
 5   free_throw_attempt_rate       21553 non-null  float64
 6   offensive_rebound_percentage  21553 non-null  float64
 7   defensive_rebound_percentage  21553 non-null  float64
 8   total_rebound_percentage      21553 non-null  float64
 9   assist_percentage             21553 non-null  float64
 10  steal_percentage              21553 non-null  float64
 11  block_percentage              21553 non-null  float64
 12  usage_percentage              21553 non-null  float64
 13 

In [234]:
#Write cleaned-up dataset to file
advanced_filtered_df.to_csv("resources/advanced_stats_filtered.csv", sep=',')
team_summary_filtered_df.to_csv("resources/team_summary_filtered.csv", sep=',')
player_totals_filtered_df.to_csv("resources/player_totals_filtered.csv", sep=',')